# Modeling

## Model 1 CNN 

CNN(Convolutional Neural Networks) are consists of five components:

- The input layer requires data inputted for preprocessing operations,  therefore, we transform the data shape from 3 dimension to 4 dimension. 
- The convolution layer introduce a concept of local perception, which tells that in the process of the human brain recognizing a picture, the whole picture is not recognized at the same time, but each feature in the picture is first perceived locally, and then the local operation is comprehensively performed at a higher level to obtain global information. We construct a CNN model that has three convolution layer, the first two convolution layer use 16 filters and the kernel size is 5 * 5. And the third convolution layer adopt one 1 * 1 kernel size aiming to reduce the dimension of image.
- The activation layer makes a non-linear mapping of the output of the convolution layer. In this model, we choose the ReLU function, because iteration is faster.
- The pooling layer mainly used for feature dimension reduction, compressing the number of data and parameters, reducing overfitting, and improving the fault tolerance of the model. We choose a max pooling over a (2, 2) window. Finally, the feature maps are converted to a 1-dimensional vector by up sampling and the third convolution layer.
- The output layer which also called fully connected layer. After several previous convolutions, activations, and pooling, the model will fully learn a high-quality feature picture fully connected layer.

CNN can directly process grayscale images and can be used directly to process image-based classification. CNN has unique advantages in image processing due to its special structure of local weight sharing. The layout is closer to the real biological neural network. Weight sharing reduces the complexity of the network, especially for multidimensional input vectors. The feature that allows images to be directly entered on the network avoids the complexity of rebuilding data when retrieving and classifying features.Maybe the performance of CNN used for MRI processing is  not well.

In [ ]:
# 定义网络结构
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #  conv1层，输入的灰度图，所以 in_channels=1, out_channels=6 说明使用了6个滤波器/卷积核，
        # kernel_size=5卷积核大小5x5
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5 ,stride = 1,padding = 2)
        # conv2层， 输入通道in_channels 要等于上一层的 out_channels
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5,stride = 1,padding = 2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1,stride = 1)
        # an affine operarion: y = Wx + b
        # 全连接层fc1,因为32x32图像输入到fc1层时候，feature map为： 5x5x16
        # 因此，全连接层的输入特征维度为16*5*5，  因为上一层conv2的out_channels=16
        # out_features=84,输出维度为84，代表该层为84个神经元
        self.upsample = nn.Upsample(scale_factor=4, mode='bilinear',align_corners=False)

#         self.fc1 = nn.Linear(1*32*80*80, 120)
#         self.fc2 = nn.Linear(in_features=120, out_features=84)
#         self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # 特征图转换为一个１维的向量
        x = self.upsample(x)
        x = self.conv3(x)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]     # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


model = Net()
#print(model)
# Try a random 32x32 input

# input = torch.randn(1, 1, 320, 320)
# out = model(input)
# print(out.shape)




# loss_func = nn.CrossEntropyLoss()
# opt = torch.optim.Adam(model.parameters(),lr=0.001)

loss_func = nn.MSELoss()
opt = torch.optim.SGD(model.parameters(), lr=0.0001)

## Model 2 U-NET

U-NET model are mainly composed of two parts :

- Feature extraction part is also called the down sampling. Every time we go through the pooling layer, we think that the scale is increased by one. Firstly, each convolutional layer contains two convolution operations, and then we encapsulate these two convolution layers into a class ConvBlock. Specifically, each convolution layer followed by kernel, instance normalization, ReLU activation and drop out, kernel size is 3 * 3 and the number of channels doubles after the first convolution operation, and remains the same the second time. Dropout is often used to randomly delete some neurons in a neural network to prevent overfitting and optimize the model by using normalization at the same time. After two convolution layers, We choose a max pooling over a (2, 2) window used to process feature map. The number of pool layers depends on experimenters.
- Up sampling is a critical part of U-net. Up sampling requires the feature map merged at the same scale as the number of channels corresponding to the feature extraction part. Therefore, before merging, the model will copy and crop the feature map in the shallow layers, which results in the final output is a region in the center of the input. And then passing through ConvBlock. Finally, in the last layer, we use the kernel 1 * 1 to accept the output image.

U-NET is derived from CNN. U-shaped structure is the biggest feature of U-NET. U-NET performs several down sampling and use the contact connection on the corresponding layer, which ensured that the finally recovered feature maps incorporated more low-level features. Multiple up sampling also makes the information such as edge restoration of the image more detailed.

In [ ]:
class ConvBlock(nn.Module):
    """
    A Convolutional Block that consists of two convolution layers each followed by
    instance normalization, relu activation and dropout.
    """

    def __init__(self, in_chans, out_chans, drop_prob):
        """
        Args:
            in_chans (int): Number of channels in the input.
            out_chans (int): Number of channels in the output.
            drop_prob (float): Dropout probability.
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.drop_prob = drop_prob

        self.layers = nn.Sequential(
            nn.Conv2d(in_chans, out_chans, kernel_size=3, padding=1),
            nn.InstanceNorm2d(out_chans),
            nn.ReLU(),
            nn.Dropout2d(drop_prob),
            nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1),
            nn.InstanceNorm2d(out_chans),
            nn.ReLU(),
            nn.Dropout2d(drop_prob)
        )

    def forward(self, input):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]

        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        return self.layers(input)

    def __repr__(self):
        return f'ConvBlock(in_chans={self.in_chans}, out_chans={self.out_chans}, ' \
            f'drop_prob={self.drop_prob})'


class UnetModel(nn.Module):
    """
    PyTorch implementation of a U-Net model.

    This is based on:
        Olaf Ronneberger, Philipp Fischer, and Thomas Brox. U-net: Convolutional networks
        for biomedical image segmentation. In International Conference on Medical image
        computing and computer-assisted intervention, pages 234–241. Springer, 2015.
    """

    def __init__(self, in_chans, out_chans, chans, num_pool_layers, drop_prob):
        """
        Args:
            in_chans (int): Number of channels in the input to the U-Net model.
            out_chans (int): Number of channels in the output to the U-Net model.
            chans (int): Number of output channels of the first convolution layer.
            num_pool_layers (int): Number of down-sampling and up-sampling layers.
            drop_prob (float): Dropout probability.
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.chans = chans
        self.num_pool_layers = num_pool_layers
        self.drop_prob = drop_prob

        self.down_sample_layers = nn.ModuleList([ConvBlock(in_chans, chans, drop_prob)])
        ch = chans
        for i in range(num_pool_layers - 1):
            self.down_sample_layers += [ConvBlock(ch, ch * 2, drop_prob)]
            ch *= 2
        self.conv = ConvBlock(ch, ch, drop_prob)

        self.up_sample_layers = nn.ModuleList()
        for i in range(num_pool_layers - 1):
            self.up_sample_layers += [ConvBlock(ch * 2, ch // 2, drop_prob)]
            ch //= 2
        self.up_sample_layers += [ConvBlock(ch * 2, ch, drop_prob)]
        self.conv2 = nn.Sequential(
            nn.Conv2d(ch, ch // 2, kernel_size=1),
            nn.Conv2d(ch // 2, out_chans, kernel_size=1),
            nn.Conv2d(out_chans, out_chans, kernel_size=1),
        )

    def forward(self, input):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]

        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        stack = []
        output = input
        # Apply down-sampling layers
        for layer in self.down_sample_layers:
            output = layer(output)
            stack.append(output)
            output = F.max_pool2d(output, kernel_size=2)

        output = self.conv(output)

        # Apply up-sampling layers
        for layer in self.up_sample_layers:
            output = F.interpolate(output, scale_factor=2, mode='bilinear', align_corners=False)
            output = torch.cat([output, stack.pop()], dim=1)
            output = layer(output)
        return self.conv2(output)
